In [4]:
import rasterio
from rasterio.enums import Resampling
import numpy as np

def create_max_value_raster(raster_files, output_raster):
    """Create a raster with the maximum value for each cell from a list of raster files.

    Args:
        raster_files (list): List of paths to raster files.
        output_raster (str): Path to the output raster file.

    Returns:
        None
    """
    # Read the first raster to initialize the max_data array and metadata
    with rasterio.open(raster_files[0]) as src0:
        meta = src0.meta.copy()
        max_data = src0.read(1)  # Read only the first band

    # Iterate over the remaining rasters
    for raster in raster_files[1:]:
        with rasterio.open(raster) as src:
            data = src.read(1)  # Read only the first band
            # Update the max_data array with the maximum values
            max_data = np.maximum(max_data, data, where=~np.isnan(data))

    # Update the metadata to reflect the number of bands and data type
    meta.update(dtype=rasterio.float32, count=1)

    # Ensure the no-data values are handled properly
    if 'nodata' in meta:
        max_data[np.isnan(max_data)] = meta['nodata']
    else:
        meta['nodata'] = -9999  # Define a no-data value if it wasn't set
        max_data[np.isnan(max_data)] = meta['nodata']

    # Write the output raster file
    with rasterio.open(output_raster, 'w', **meta) as dst:
        dst.write(max_data, 1)
# Example usage:
raster_files = [
r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Precipitation\max_60min_intensity_050121_to_070921_clipped.tif",
r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Precipitation\max_60min_intensity_050122_to_070922_clipped.tif",
r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Precipitation\max_60min_intensity_050123_to_070923_clipped.tif",
r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Precipitation\max_60min_intensity_070923_to_102123_clipped.tif",
r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Precipitation\max_60min_intensity_071021_to_103121_clipped.tif",
r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Precipitation\max_60min_intensity_071022_to_103122_clipped.tif",]
output_raster = r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Precipitation\max_60min_intensity_050121_to_102123.tif"

create_max_value_raster(raster_files, output_raster)


In [5]:
def sum_raster_values(raster_files, output_raster):
    """Sum the values of each cell across multiple rasters and save the result to a new raster.

    Args:
        raster_files (list): List of paths to raster files.
        output_raster (str): Path to the output raster file.

    Returns:
        None
    """
    # Open the first raster to initialize the sum_data array and metadata
    with rasterio.open(raster_files[0]) as src0:
        meta = src0.meta.copy()
        # Start with the first raster's data to initialize sum_data
        sum_data = src0.read(1).astype(np.float32)  # Ensure float type for handling potential large values

    # Iterate over the remaining rasters
    for raster in raster_files[1:]:
        with rasterio.open(raster) as src:
            data = src.read(1).astype(np.float32)
            # Use numpy to sum up the data arrays
            sum_data += data

    # Update the metadata to reflect the data type and the number of bands
    meta.update(dtype=rasterio.float32, count=1)

    # Handle no-data values appropriately
    if 'nodata' in meta:
        sum_data[np.isnan(sum_data)] = meta['nodata']
    else:
        meta['nodata'] = -9999  # Define a no-data value if it wasn't set
        sum_data[np.isnan(sum_data)] = meta['nodata']

    # Write the output raster file
    with rasterio.open(output_raster, 'w', **meta) as dst:
        dst.write(sum_data, 1)
        
raster_files = [
r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Precipitation\running_total_050121_to_070921_clipped.tif",
r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Precipitation\running_total_050122_to_070922_clipped.tif",
r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Precipitation\running_total_050123_to_070923_clipped.tif",
r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Precipitation\running_total_070923_to_102123_clipped.tif",
r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Precipitation\running_total_071021_to_103121_clipped.tif",
r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Precipitation\running_total_071022_to_103122_clipped.tif"
]

outfile = r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Precipitation\running_total_050121_to_103123.tif"

sum_raster_values(raster_files, outfile)

$$ \delta_{\text{DoD}}^{i,j} = \pm 1.96 \times \sqrt{\delta_{\text{LIDAR}} + \delta_{\text{SfM}}^{i,j}} $$

where 
$$ \delta_{\text{LIDAR}} = 0.12 \text{ meters} $$
$$ \delta_{\text{SfM}}^{i,j} = \text{Krigged error from Metashape precision estimates} $$

